In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
data= pd.read_csv('sep19SexAndAgeAddedFINAL DATASET_ver2.txt') 
print(data.shape)


(535169, 406)


In [3]:
#Filter the data

#remove diabetic people 
data=data[data.FIELD_16!=1]
data=data[data.FIELD_23!=1]

#remove hypertensive people who are diagnosed and on drugs
# data=data[data.FIELD_15!=1]
# data=data[data.FIELD_22!=1]

#remove people who are diagnosed and on drugs for high lipids  
# data=data[data.FIELD_17!=1]
# data=data[data.FIELD_24!=1]

print(data.shape)

(516594, 406)


In [4]:
newdata=data.copy()
print(newdata.shape)
newdata.head()

(516594, 406)


,Unnamed: 0,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S004423,S004424,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102
0,0,1000003,20130913,100000320130913,46.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,14.0
1,1,1000003,20140613,100000320140613,47.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,16.0
2,2,1000003,20150917,100000320150917,48.0,670212.0,0.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0
3,3,1000003,20160617,100000320160617,49.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0
4,4,1000003,20170608,100000320170608,50.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0


In [5]:
# for checking if there are two records per day
newdata_count_temp=newdata.groupby(by=['FIELD_1','FIELD_2']).size().reset_index(name='freq') 
# print(newdata_count_temp[newdata_count_temp.freq>1])

# group records by id and year to find out the number of tests per specific year
newdata_count_temp['Year']=(newdata_count_temp.FIELD_2 / 10000).astype(int)
newdata_count_peryear=newdata_count_temp.groupby(by=['FIELD_1','Year']).size().reset_index(name='freq')
# max(temp[temp.freq>1].freq)
newdata_count_peryear=newdata_count_peryear.sort_values(by='FIELD_1')
newdata_count_peryear.head()

# to get the number of years a patient visited
newdata_count_peryearG1= newdata_count_peryear.groupby(by=['FIELD_1']).size().reset_index(name='freq')
newdata_count_peryearG1.head()
print(newdata_count_peryearG1.freq.unique())
newdata_count_peryearG1=newdata_count_peryearG1[newdata_count_peryearG1.freq>1]
print(newdata_count_peryearG1.shape)
newdata_count_peryearG1.head()


[1 4 2 5 3 6]
(117617, 2)


,FIELD_1,freq
1,64,4
3,105,2
4,323,4
8,761,2
9,782,2


In [6]:
mylist=[]
for i in newdata_count_peryearG1.FIELD_1:
    vals=newdata_count_peryear[newdata_count_peryear.FIELD_1==i]
    mylist.append([i,np.sort(list(vals.Year))])

In [7]:
mylistPD=pd.DataFrame(mylist,columns=['FIELD_1','Years'])
print(mylistPD.shape)
mylistPD.tail(10)

(117617, 2)


,FIELD_1,Years
117607,5542037,"[2014, 2018]"
117608,5595210,"[2014, 2018]"
117609,5654226,"[2014, 2015, 2016]"
117610,5816083,"[2016, 2018]"
117611,5825127,"[2016, 2018]"
117612,5867953,"[2016, 2017]"
117613,5884342,"[2014, 2015]"
117614,5885504,"[2016, 2017]"
117615,5887822,"[2016, 2017]"
117616,5887825,"[2016, 2017]"


In [8]:
newdata['Year']=(newdata.FIELD_2 / 10000).astype(int)

x=[]
target=[]

for _id in mylistPD.FIELD_1:
    ID=_id
    years=mylistPD[mylistPD.FIELD_1==ID].Years.values

    for y in range(len(years[0])-1):
        dif=years[0][y+1] - years[0][y]
        if(dif==1):
            
            tempdata=newdata.query('FIELD_1=='+str(ID)+' and Year=='+str(years[0][y])).reset_index()         
            x.append(tempdata.iloc[tempdata.shape[0]-1,:].values)

            tempdata=newdata.query('FIELD_1=='+str(ID)+' and Year=='+str(years[0][y+1])).reset_index()         
            target.append(tempdata.iloc[tempdata.shape[0]-1,:].values)



In [9]:
newdata.head()

,Unnamed: 0,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S004424,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102,Year
0,0,1000003,20130913,100000320130913,46.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,14.0,2013
1,1,1000003,20140613,100000320140613,47.0,670212.0,0.0,NaN,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,16.0,2014
2,2,1000003,20150917,100000320150917,48.0,670212.0,0.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0,2015
3,3,1000003,20160617,100000320160617,49.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0,2016
4,4,1000003,20170608,100000320170608,50.0,670212.0,0.0,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0,2017


In [10]:
# print(list(newdata.columns))

newcollist=newdata.columns
newcollist= np.insert(newcollist,0,'index')
newcollist

Index(['index', 'Unnamed: 0', 'FIELD_1', 'FIELD_2', 'COMPARE', 'AGE', 'B_DAY',
       'SEX', 'FIELD_3', 'FIELD_4',
       ...
       'S004424', 'S005200', 'S007400', 'S008501', 'S008502', 'S008503',
       'S008504', 'S010101', 'S010102', 'Year'],
      dtype='object', length=408)

In [11]:
newdata.shape
tempcols=newcollist
# print(list(newdata.columns))

In [12]:
xlable=pd.DataFrame(x,columns=tempcols).drop(columns='Unnamed: 0')
xlable.tail(10)

,index,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S004424,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102,Year
185833,383358,5528601,20171216,552860120171216,35.0,820223.0,0.0,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,2017
185834,383699,5529756,20171219,552975620171219,37.0,800701.0,0.0,1,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
185835,384801,5534268,20171228,553426820171228,48.0,691013.0,1.0,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,15.0,2017
185836,394319,5654226,20140616,565422620140616,31.0,830802.0,1.0,NaN,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,18.0,2014
185837,394320,5654226,20151125,565422620151125,32.0,830802.0,1.0,1,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,14.0,2015
185838,405353,5867953,20160718,586795320160718,38.0,781211.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,12.0,2016
185839,405670,5884342,20141023,588434220141023,55.0,590727.0,1.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,15.0,2014
185840,405696,5885504,20160923,588550420160923,34.0,820407.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,16.0,2016
185841,405745,5887822,20160831,588782220160831,44.0,721017.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,11.0,2016
185842,405747,5887825,20160930,588782520160930,44.0,721011.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,11.0,2016


In [13]:
targelable=pd.DataFrame(target,columns=tempcols).drop(columns='Unnamed: 0')
targelable.tail(10)

,index,FIELD_1,FIELD_2,COMPARE,AGE,B_DAY,SEX,FIELD_3,FIELD_4,FIELD_5,...,S004424,S005200,S007400,S008501,S008502,S008503,S008504,S010101,S010102,Year
185833,383359,5528601,20180501,552860120180501,36.0,820223.0,0.0,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,10.0,2018
185834,383700,5529756,20180523,552975620180523,38.0,800701.0,0.0,1,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,17.0,2018
185835,384802,5534268,20180618,553426820180618,49.0,691013.0,1.0,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,14.0,2018
185836,394320,5654226,20151125,565422620151125,32.0,830802.0,1.0,1,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,14.0,2015
185837,394321,5654226,20160826,565422620160826,33.0,830802.0,1.0,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,15.0,2016
185838,405354,5867953,20170719,586795320170719,39.0,781211.0,0.0,NaN,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,13.0,2017
185839,405671,5884342,20151112,588434220151112,56.0,590727.0,1.0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,16.0,2015
185840,405697,5885504,20170703,588550420170703,35.0,820407.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,18.0,2017
185841,405746,5887822,20170816,588782220170816,45.0,721017.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,11.0,2017
185842,405748,5887825,20170816,588782520170816,45.0,721011.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,11.0,2017


In [14]:
print(xlable.shape)
print(targelable.shape)

(185843, 407)
(185843, 407)


In [16]:
# targelable.to_csv('_targelable4_withNa_AllColumns_OnDrugForLipid.txt',sep=',')
# xlable.to_csv('_xlable4_withNa_AllColumns_OnDrugForLipid.txt',sep=',')

In [15]:
targelable.to_csv('TargetLable_onlyDiabeticRemoved.txt',sep=',')
xlable.to_csv('XLable_onlyDiabeticRemoved.txt',sep=',')

In [ ]:
print(xlable.shape)
print(targelable.shape)